# Mundo 6 - API do IBGE

A API do IBGE é meio lixo supremo, mas da pra tirar algumas coisas interessantes.


### Conceitos importantes de hoje:

 * APIs com endpoints diferentes
 * Tratamento de dados

### 2 informações importantes obtidas pela API:
 
 * Projeção populacional por país ou regiões
 * Indicadores econômicos por país
 
### Documentação geral

https://servicodados.ibge.gov.br/api/docs


# Projeção populacional por país ou regiões

### Documentação:

https://servicodados.ibge.gov.br/api/docs/projecoes

## URL Base

https://servicodados.ibge.gov.br/api/v1/projecoes/populacao/{localidade}

Localidade deve ser:

    * País: BR
    * Regiões: 
        ** 1 - Norte
        ** 2 - Nordeste
        ** 3 - Sudeste
        ** 4 - Sul
        ** 5 - Centro-Oeste

In [1]:
import requests
import pandas as pd
import json

In [2]:
localidade = 'BR'
url = f'https://servicodados.ibge.gov.br/api/v1/projecoes/populacao/{localidade}'

response = requests.get(url)
data = json.loads(response.text)
data

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Indicadores econômicos por país

### Documentação: 

https://servicodados.ibge.gov.br/api/docs/paises

## URL Base

https://servicodados.ibge.gov.br/api/v1/paises/{paises}/indicadores/{indicadores}


obs: Quando desejar mais de um indicador, ou país, utilizar o "|"

Localidade deve ser:

    * País: Código de 2 digitos de acordo com  norma ISO 3166-1 ALPHA-2 (Exemplo: BR, US)

Indicador deve ser:

    * Um número de acordo com o site do IBGE (ele define id's)

obs: Na documentação tem o id dos indicadores e o código de cada país

In [4]:
paises = 'BR|US'
indicadores = '77823' #PIB per capita
nome_pais = ['Brasil', 'EUA']


url = f'https://servicodados.ibge.gov.br/api/v1/paises/{paises}/indicadores/{indicadores}'

response = requests.get(url)
data = json.loads(response.text)

lista_dfs = []

for i, nome_pais in enumerate(nome_pais):
    
    lista_anos = []
    lista_valores = []

    for informacoes in data[0]['series'][i]['serie']: #no lugar desse "0", caso você puxe dois indicadores ao mesmo tempo, vai entrar um outro loop.

        valores = list(informacoes.items())
        lista_anos.append(valores[0][0])
        lista_valores.append(valores[0][1])

    df = pd.DataFrame(list(zip(lista_anos,lista_valores)), columns=["Anos",f"{nome_pais}"]).dropna()

    lista_dfs.append(df)
    
df_final = pd.merge(lista_dfs[0],lista_dfs[1], on='Anos')
df_final = df_final.set_index("Anos")
df_final = df_final.applymap(lambda x: int(x))

display(df_final)

/var/folders/49/ds_8p60s0c13tflg2l28s78w0000gp/T/ipykernel_34144/872853588.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_final = df_final.applymap(lambda x: int(x))


ValueError: invalid literal for int() with base 10: '23888.60'

In [ ]:
data

In [ ]:
df_final['eua_maior'] = df_final['EUA']/df_final['Brasil']

df_final

# Exercícios 

* Exercício 139: Colete os dados de indivíduos com acesso a internet no BR na API de indicadores.
* Exercício 140: Colete os nomes mais registrados no Brasil desde 1950.

In [ ]:
#gabarito 139

paises = 'BR'
indicadores = '77857' #Pessoas com internet

url = f'https://servicodados.ibge.gov.br/api/v1/paises/{paises}/indicadores/{indicadores}'

response = requests.get(url)
data = json.loads(response.text)

lista_dfs = []    
lista_anos = []
lista_valores = []

for informacoes in data[0]['series'][0]['serie']: 
        
        #no lugar desse "0", caso você puxe dois indicadores ao mesmo tempo, vai entrar um outro loop.

        valores = list(informacoes.items())
        lista_anos.append(valores[0][0])
        lista_valores.append(valores[0][1])

df = pd.DataFrame(list(zip(lista_anos,lista_valores)), columns=["Anos", "Brasil"]).dropna()
df = df.set_index("Anos")

print(df)

In [ ]:
#gabarito 140 

url = 'https://servicodados.ibge.gov.br/api/v2/censos/nomes/ranking'

response = requests.get(url)
data = json.loads(response.text)

lista_df = []

for informacoes in data[0]['res']: 

    df = pd.DataFrame(informacoes, index = [0])
    
    lista_df.append(df)
    
rankings_nomes = pd.concat(lista_df, ignore_index = True)

rankings_nomes